In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import figure1

In [ ]:
VERSION = 1
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig1/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Load data
# Load an example session
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH8.2h_S20230116'
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)


# Get PSTH for all sessions
project='hey2_3d-vision_foodres_20220101'
flexilims_session = flz.get_flexilims_session(project)
session_list = [
                "PZAH6.4b_S20220419", 
                "PZAH6.4b_S20220426"
                ]
results_all = figure1.get_psth_crossval_all_sessions(flexilims_session, session_list, nbins=10)

In [ ]:
fig = plt.figure()

# Fig.1A: Motion parallax schema & equation, VR schema
# Fig.1B: Trial structure schema, Spheres with different sizes schema

# Fig.1C: Raster plot of an example neuron
EXAMPLE_ROI = 250
figure1.plot_raster_all_depths(
    fig=fig,
    neurons_df=neurons_df_example,
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    max_distance=6,
    nbins=60,
    frame_rate=15,
    vmax=3,
    plot=True,
    plot_x=0,
    plot_y=1,
    plot_width=0.8,
    plot_height=0.2,
    cbar_width=0.01,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
    );

# Fig.1D: Example PSTH of a single neuron
figure1.plot_PSTH(fig=fig,
            trials_df=trials_df_example,
            roi=EXAMPLE_ROI,
            is_closed_loop=True,
            max_distance=6,
            nbins=20,
            frame_rate=15,
            plot_x=0.85,
            plot_y=1,
            plot_width=0.2,
            plot_height=0.2,
            fontsize_dict={'title':15, 'label':10, 'tick':10},
            linewidth=3,
            )

# Fig.1E: Example depth tuning curve & PSTH of a single neuron
figure1.plot_depth_tuning_curve(
    fig=fig,
    neurons_df=neurons_df_example,
    trials_df=trials_df_example,
    roi=250,
    rs_thr=0.2,
    plot_fit=False,
    linewidth=3,
    linecolor="k",
    fit_linecolor="r",
    closed_loop=1,
    plot_x=0,
    plot_y=0.67,
    plot_width=0.2,
    plot_height=0.2,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
    )

# Fig.1F: Example depth tuning curve & PSTH of 3 other neurons
EXAMPLE_ROIS = [249, 54, 88]
for i, roi in enumerate(EXAMPLE_ROIS):
    figure1.plot_PSTH(fig=fig,
            trials_df=trials_df_example,
            roi=roi,
            is_closed_loop=True,
            max_distance=6,
            nbins=20,
            frame_rate=15,
            plot_x=0.05+0.25*(i+1),
            plot_y=0.67,
            plot_width=0.2,
            plot_height=0.18,
            fontsize_dict={'title':15, 'label':10, 'tick':10},
            linewidth=2,
            )
    if i!=0:
        plt.ylabel('')
    
    figure1.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.05+0.25*(i+1)+0.12,
        plot_y=0.79,
        plot_width=0.08,
        plot_height=0.08,
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks([])
        
# Fig.1G: Histogram of preferred depths of all depth-tuned neurons; Raster plot of preferred depths of all neurons (sorted by preferred depth)
figure1.plot_preferred_depth_hist(
    fig=fig,
    results_df=results_all,
    use_col="preferred_depth_crossval",
    nbins=50,
    plot_x=0,
    plot_y=0.2,
    plot_width=0.4,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
)

figure1.plot_psth_raster(
    fig=fig,
    results_df=results_all,
    depth_list=np.geomspace(6,600,num=5),
    use_cols=["preferred_depth_crossval", "psth_crossval", "preferred_depth_rsq"],
    depth_rsq_thr = 0.04,
    plot_x=0.6,
    plot_y=0.2,
    plot_width=0.4,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
)

fig.savefig(SAVE_ROOT+'fig1.pdf', bbox_inches='tight', transparent=True)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg

# Load the image
img = mpimg.imread('plot_image.png')

# Create a figure
fig, ax = plt.subplots()

# Display the image
ax.imshow(img)
ax.axis('off')  # Turn off axis

plt.show()